# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 15 2022 1:09PM,253202,12,HH-38772,Hush Hush,Released
1,Dec 15 2022 1:07PM,253201,10,MTR-22-0279,"Senseonics, Incorporated",Released
2,Dec 15 2022 1:04PM,253200,12,HH8234.2,Hush Hush,Released
3,Dec 15 2022 12:42PM,253199,12,8906087,"SD Head USA, LLC",Executing
4,Dec 15 2022 12:15PM,253197,10,MSP216812,Methapharm-G,Released
5,Dec 15 2022 12:14PM,253193,10,0086251015,ISDIN Corporation,Released
6,Dec 15 2022 12:14PM,253193,10,0086251021,ISDIN Corporation,Released
7,Dec 15 2022 12:14PM,253193,10,0086251016,ISDIN Corporation,Released
8,Dec 15 2022 12:14PM,253193,10,0086251026,ISDIN Corporation,Released
9,Dec 15 2022 12:14PM,253193,10,0086251061,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,253197,Released,1
31,253199,Executing,1
32,253200,Released,1
33,253201,Released,1
34,253202,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
253197,NaN,NaN,NaN,1.0
253199,NaN,NaN,1.0,NaN
253200,NaN,NaN,NaN,1.0
253201,NaN,NaN,NaN,1.0
253202,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
253117,0.0,0.0,0.0,1.0
253125,0.0,0.0,0.0,1.0
253126,0.0,0.0,0.0,1.0
253138,0.0,0.0,0.0,1.0
253139,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
253117,0.0,0,0,1
253125,0.0,0,0,1
253126,0.0,0,0,1
253138,0.0,0,0,1
253139,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,253117,0.0,0,0,1
1,253125,0.0,0,0,1
2,253126,0.0,0,0,1
3,253138,0.0,0,0,1
4,253139,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,253117,0.0,,,1
1,253125,0.0,,,1
2,253126,0.0,,,1
3,253138,0.0,,,1
4,253139,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 15 2022 1:09PM,253202,12,Hush Hush
1,Dec 15 2022 1:07PM,253201,10,"Senseonics, Incorporated"
2,Dec 15 2022 1:04PM,253200,12,Hush Hush
3,Dec 15 2022 12:42PM,253199,12,"SD Head USA, LLC"
4,Dec 15 2022 12:15PM,253197,10,Methapharm-G
5,Dec 15 2022 12:14PM,253193,10,ISDIN Corporation
11,Dec 15 2022 12:14PM,253188,10,ISDIN Corporation
28,Dec 15 2022 12:12PM,253196,21,"NBTY Global, Inc."
29,Dec 15 2022 12:10PM,253195,21,"NBTY Global, Inc."
30,Dec 15 2022 12:08PM,253194,10,"Virtue Rx, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Dec 15 2022 1:09PM,253202,12,Hush Hush,0.0,,,1
1,Dec 15 2022 1:07PM,253201,10,"Senseonics, Incorporated",0.0,,,1
2,Dec 15 2022 1:04PM,253200,12,Hush Hush,0.0,,,1
3,Dec 15 2022 12:42PM,253199,12,"SD Head USA, LLC",0.0,,1,
4,Dec 15 2022 12:15PM,253197,10,Methapharm-G,0.0,,,1
5,Dec 15 2022 12:14PM,253193,10,ISDIN Corporation,0.0,,,6
6,Dec 15 2022 12:14PM,253188,10,ISDIN Corporation,0.0,,,17
7,Dec 15 2022 12:12PM,253196,21,"NBTY Global, Inc.",0.0,,,1
8,Dec 15 2022 12:10PM,253195,21,"NBTY Global, Inc.",0.0,,,1
9,Dec 15 2022 12:08PM,253194,10,"Virtue Rx, LLC",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 1:09PM,253202,12,Hush Hush,1,,
1,Dec 15 2022 1:07PM,253201,10,"Senseonics, Incorporated",1,,
2,Dec 15 2022 1:04PM,253200,12,Hush Hush,1,,
3,Dec 15 2022 12:42PM,253199,12,"SD Head USA, LLC",,1,
4,Dec 15 2022 12:15PM,253197,10,Methapharm-G,1,,
5,Dec 15 2022 12:14PM,253193,10,ISDIN Corporation,6,,
6,Dec 15 2022 12:14PM,253188,10,ISDIN Corporation,17,,
7,Dec 15 2022 12:12PM,253196,21,"NBTY Global, Inc.",1,,
8,Dec 15 2022 12:10PM,253195,21,"NBTY Global, Inc.",1,,
9,Dec 15 2022 12:08PM,253194,10,"Virtue Rx, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 1:09PM,253202,12,Hush Hush,1,,
1,Dec 15 2022 1:07PM,253201,10,"Senseonics, Incorporated",1,,
2,Dec 15 2022 1:04PM,253200,12,Hush Hush,1,,
3,Dec 15 2022 12:42PM,253199,12,"SD Head USA, LLC",,1,
4,Dec 15 2022 12:15PM,253197,10,Methapharm-G,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 1:09PM,253202,12,Hush Hush,1.0,NaN,NaN
1,Dec 15 2022 1:07PM,253201,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Dec 15 2022 1:04PM,253200,12,Hush Hush,1.0,NaN,NaN
3,Dec 15 2022 12:42PM,253199,12,"SD Head USA, LLC",NaN,1.0,NaN
4,Dec 15 2022 12:15PM,253197,10,Methapharm-G,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 1:09PM,253202,12,Hush Hush,1.0,0.0,0.0
1,Dec 15 2022 1:07PM,253201,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Dec 15 2022 1:04PM,253200,12,Hush Hush,1.0,0.0,0.0
3,Dec 15 2022 12:42PM,253199,12,"SD Head USA, LLC",0.0,1.0,0.0
4,Dec 15 2022 12:15PM,253197,10,Methapharm-G,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3544525,104.0,53.0,13.0
12,759601,2.0,1.0,0.0
15,253117,1.0,0.0,0.0
16,759485,3.0,0.0,0.0
19,1265767,24.0,11.0,0.0
20,253139,1.0,0.0,0.0
21,506391,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3544525,104.0,53.0,13.0
1,12,759601,2.0,1.0,0.0
2,15,253117,1.0,0.0,0.0
3,16,759485,3.0,0.0,0.0
4,19,1265767,24.0,11.0,0.0
5,20,253139,1.0,0.0,0.0
6,21,506391,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,104.0,53.0,13.0
1,12,2.0,1.0,0.0
2,15,1.0,0.0,0.0
3,16,3.0,0.0,0.0
4,19,24.0,11.0,0.0
5,20,1.0,0.0,0.0
6,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,104.0
1,12,Released,2.0
2,15,Released,1.0
3,16,Released,3.0
4,19,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,13.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,53.0,1.0,0.0,0.0,11.0,0.0,0.0
Released,104.0,2.0,1.0,3.0,24.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,13.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,53.0,1.0,0.0,0.0,11.0,0.0,0.0
2,Released,104.0,2.0,1.0,3.0,24.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,13.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,53.0,1.0,0.0,0.0,11.0,0.0,0.0
2,Released,104.0,2.0,1.0,3.0,24.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()